<a href="https://colab.research.google.com/github/GUOJL23/COMP90089/blob/main/DataExtractionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
project_id = 'vast-arena-431203-r0'

In [ ]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
# Authenticating to use bigquery
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# test it works
df = run_query("""
SELECT subject_id
FROM `physionet-data.mimiciv_hosp.patients`
WHERE subject_id = 10012853
""")
assert df.shape[0] == 1, 'unable to query MIMIC-IV!'

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [ ]:
# Query to get itemid for creatinine in labitems table
creatinine_query = """
    SELECT itemid, label
    FROM `physionet-data.mimiciv_hosp.d_labitems`
    WHERE label LIKE '%Creatinine%';
"""
creatinine_df = run_query(creatinine_query)
creatinine_df

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,itemid,label
0,52024,"Creatinine, Whole Blood"
1,50912,Creatinine
2,52546,Creatinine
3,51937,"Creatinine, Stool"
4,51067,24 hr Creatinine
5,51070,"Albumin/Creatinine, Urine"
6,51073,"Amylase/Creatinine Ratio, Urine"
7,51080,Creatinine Clearance
8,51081,"Creatinine, Serum"
9,51082,"Creatinine, Urine"


In [ ]:
# query to get simple creatinine level data from patients
creatinine_query_icu = run_query("""
  SELECT subject_id, charttime, safe_cast(value AS FLOAT64) as creatinine_level
  FROM `physionet-data.mimiciv_hosp.labevents`
  WHERE itemid in (51081,50912,51977)
""")
creatinine_query_icu

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,charttime,creatinine_level
0,10004235,2196-02-26 06:26:00,4.0
1,10015860,2192-05-27 07:15:00,4.2
2,10041894,2143-06-29 06:50:00,3.0
3,10048244,2122-07-27 12:00:00,3.1
4,10094811,2118-06-22 07:00:00,5.3
...,...,...,...
3282898,19547030,2166-08-26 11:45:00,1.8
3282899,19566168,2160-10-06 05:15:00,1.8
3282900,19575785,2122-04-19 11:16:00,1.8
3282901,19713842,2153-08-06 04:30:00,1.8


In [ ]:
# Query to get subject_id, creatinine level and previous creatinine level of patients who had a creatinine level change of more that 0.3 mg/dL within 48 hrs or increased by 1.5 times in 7 days.
# info from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10100386/
creatinine_change_query_with_icu = run_query("""
WITH creatinine_data AS (
  SELECT
    subject_id,
    charttime,
    safe_cast(value AS FLOAT64) AS creatinine_level,
    valueuom,
    LAG(safe_cast(value AS FLOAT64)) OVER (PARTITION BY subject_id ORDER BY charttime) AS previous_creatinine_level,
    DATETIME_DIFF(charttime, LAG(charttime) OVER (PARTITION BY subject_id ORDER BY charttime), HOUR) AS time_diff_hours
  FROM `physionet-data.mimiciv_hosp.labevents`
  WHERE itemid IN (51081, 50912, 51977)
)

SELECT
  subject_id,
  creatinine_level,
  previous_creatinine_level,
  (creatinine_level - previous_creatinine_level) AS creatinine_change,
  charttime,
  CASE
    WHEN (time_diff_hours <= 48 AND (creatinine_level - previous_creatinine_level) >= 0.3) THEN '0.3 mg/dL increase in 48 hours'
    WHEN (time_diff_hours <= 168 AND (creatinine_level / NULLIF(previous_creatinine_level, 0)) >= 1.5) THEN '1.5x increase in 7 days'
    ELSE 'None'
  END AS condition_satisfied
FROM creatinine_data
WHERE
  previous_creatinine_level IS NOT NULL
  AND (
    (time_diff_hours <= 48 AND (creatinine_level - previous_creatinine_level) >= 0.3)
    OR
    (time_diff_hours <= 168 AND (creatinine_level / NULLIF(previous_creatinine_level, 0)) >= 1.5)
  )

""")
creatinine_change_query_with_icu

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,creatinine_level,previous_creatinine_level,creatinine_change,charttime,condition_satisfied
0,10112984,8.2,6.3,1.9,2160-08-05 07:20:00,0.3 mg/dL increase in 48 hours
1,10112984,9.7,8.2,1.5,2160-08-06 04:55:00,0.3 mg/dL increase in 48 hours
2,10112984,8.4,6.1,2.3,2160-08-08 05:10:00,0.3 mg/dL increase in 48 hours
3,10112984,7.8,6.7,1.1,2160-08-15 02:11:00,0.3 mg/dL increase in 48 hours
4,11020538,7.6,6.1,1.5,2180-10-31 07:20:00,0.3 mg/dL increase in 48 hours
...,...,...,...,...,...,...
105622,18978412,1.8,1.5,0.3,2128-05-07 00:01:00,0.3 mg/dL increase in 48 hours
105623,19523017,1.8,0.8,1.0,2167-04-15 16:30:00,1.5x increase in 7 days
105624,19592728,1.8,1.3,0.5,2128-06-29 05:30:00,0.3 mg/dL increase in 48 hours
105625,19735459,1.8,1.5,0.3,2132-10-01 17:53:00,0.3 mg/dL increase in 48 hours


In [ ]:
# Get subject id, and medication details of patients taking drugs that are found to cause aki
medication_query = run_query("""
  SELECT
    subject_id,
    hadm_id,
    drug,
    ndc,
    starttime,
    stoptime
FROM
    `physionet-data.mimiciv_hosp.prescriptions`
WHERE
    LOWER(drug) IN (
        'ibuprofen', 'naproxen', 'aspirin', 'diclofenac', 'indomethacin',
        'gentamicin', 'tobramycin', 'vancomycin', 'amphotericin b', 'trimethoprim-sulfamethoxazole',
        'cephalexin', 'furosemide', 'bumetanide', 'hydrochlorothiazide', 'spironolactone',
        'amiloride', 'lisinopril', 'enalapril', 'ramipril', 'captopril',
        'losartan', 'valsartan', 'irbesartan', 'candesartan', 'cisplatin',
        'carboplatin', 'methotrexate', 'ifosfamide', 'cyclosporine', 'tacrolimus',
        'sirolimus', 'acyclovir', 'foscarnet', 'tenofovir', 'cidofovir',
        'omeprazole', 'pantoprazole', 'lansoprazole', 'esomeprazole', 'allopurinol',
        'lithium'
    )
ORDER BY
    subject_id, starttime
LIMIT 1000
""")
medication_query

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,drug,ndc,starttime,stoptime
0,10000032,22595853,Spironolactone,63739054410,2180-05-07 01:00:00,2180-05-07 09:00:00
1,10000032,22595853,Furosemide,51079007220,2180-05-07 01:00:00,2180-05-07 09:00:00
2,10000032,22595853,Spironolactone,63739054410,2180-05-08 08:00:00,2180-05-07 22:00:00
3,10000032,22595853,Furosemide,51079007320,2180-05-08 08:00:00,2180-05-07 22:00:00
4,10000032,22841357,Furosemide,51079007220,2180-06-26 23:00:00,2180-06-26 22:00:00
...,...,...,...,...,...,...
995,10008924,27441295,Furosemide,00182117089,2139-04-17 20:00:00,2139-04-18 09:00:00
996,10008924,27441295,Pantoprazole,00008084199,2139-04-18 16:00:00,2139-04-22 19:00:00
997,10008924,27441295,Furosemide,00182116189,2139-04-18 20:00:00,2139-04-22 19:00:00
998,10008924,23676183,Omeprazole,00093521193,2139-07-08 20:00:00,2139-07-09 23:00:00


In [ ]:
# Medication details of patinets who satify aki criteria using result of last query
subject_ids = tuple(creatinine_change_query_with_icu['subject_id'].unique())

# New query to fetch medication details based on subject_id from previous query
medication_query_with_creatinine_change = run_query(f"""
SELECT
  subject_id,
  hadm_id,
  drug,
  ndc,
  starttime,
  stoptime
FROM
  `physionet-data.mimiciv_hosp.prescriptions`
WHERE
  subject_id IN {subject_ids}
  AND LOWER(drug) IN (
    'gentamicin', 'tobramycin', 'vancomycin', 'amphotericin b', 'trimethoprim-sulfamethoxazole',
    'cephalexin', 'furosemide', 'bumetanide', 'hydrochlorothiazide', 'spironolactone',
    'amiloride', 'lisinopril', 'enalapril', 'ramipril', 'captopril',
    'losartan', 'valsartan', 'irbesartan', 'candesartan', 'cisplatin',
    'carboplatin', 'methotrexate', 'ifosfamide', 'cyclosporine', 'tacrolimus',
    'sirolimus', 'acyclovir', 'foscarnet', 'tenofovir', 'cidofovir',
    'omeprazole', 'pantoprazole', 'lansoprazole', 'esomeprazole', 'allopurinol',
    'lithium'
  )
ORDER BY
  subject_id, starttime
""")

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [ ]:
# Assuming medication_query_with_creatinine_change contains the new medication data
medication_data = medication_query_with_creatinine_change

# Merge the creatinine change data with medication data
merged_data = pd.merge(
    medication_data,
    creatinine_change_query_with_icu,
    on='subject_id',  # You can join on subject_id
    how='inner'
)

# Ensure that 'charttime' and 'starttime' are in datetime format
merged_data['charttime'] = pd.to_datetime(merged_data['charttime'])
merged_data['starttime'] = pd.to_datetime(merged_data['starttime'])

# Calculate the difference in days and add a new column 'days_prescribed_ahead'
merged_data['days_prescribed_ahead'] = (merged_data['charttime'] - merged_data['starttime']).dt.days

# Filter medications prescribed before the creatinine charttime
filtered_data = merged_data[merged_data['starttime'] < merged_data['charttime']]

In [ ]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3369206 entries, 0 to 5797434
Data columns (total 12 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   subject_id                 Int64         
 1   hadm_id                    Int64         
 2   drug                       object        
 3   ndc                        object        
 4   starttime                  datetime64[us]
 5   stoptime                   datetime64[us]
 6   creatinine_level           float64       
 7   previous_creatinine_level  float64       
 8   creatinine_change          float64       
 9   charttime                  datetime64[us]
 10  condition_satisfied        object        
 11  days_prescribed_ahead      float64       
dtypes: Int64(2), datetime64[us](3), float64(4), object(3)
memory usage: 340.6+ MB


In [ ]:
demographics_query = run_query(f"""
SELECT
    subject_id,
    gender,
    anchor_age
FROM
    `physionet-data.mimiciv_hosp.patients`
WHERE
    subject_id IN {subject_ids}
ORDER BY
    subject_id
""")
demographics_query

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,gender,anchor_age
0,10000032,F,52
1,10000935,F,52
2,10000980,F,73
3,10001338,F,43
4,10002013,F,53
...,...,...,...
28863,19997538,M,53
28864,19997752,F,66
28865,19998330,F,71
28866,19998497,F,82


In [ ]:
merged_data = pd.merge(
    merged_data,           # Use the DataFrame that contains medication and creatinine change data
    demographics_query,      # New demographics data
    on='subject_id',        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5797439 entries, 0 to 5797438
Data columns (total 14 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   subject_id                 Int64         
 1   hadm_id                    Int64         
 2   drug                       object        
 3   ndc                        object        
 4   starttime                  datetime64[us]
 5   stoptime                   datetime64[us]
 6   creatinine_level           float64       
 7   previous_creatinine_level  float64       
 8   creatinine_change          float64       
 9   charttime                  datetime64[us]
 10  condition_satisfied        object        
 11  days_prescribed_ahead      float64       
 12  gender                     object        
 13  anchor_age                 Int64         
dtypes: Int64(3), datetime64[us](3), float64(4), object(4)
memory usage: 635.8+ MB


In [ ]:
race_query = run_query(f"""
SELECT
    distinct(subject_id),
    race
FROM
    `physionet-data.mimiciv_hosp.admissions`
WHERE
    subject_id IN {subject_ids}
ORDER BY
    subject_id
""")
race_query

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,race
0,10000032,WHITE
1,10000935,BLACK/AFRICAN AMERICAN
2,10000980,BLACK/AFRICAN AMERICAN
3,10001338,WHITE
4,10002013,WHITE
...,...,...
31499,19997538,WHITE
31500,19997752,WHITE
31501,19998330,BLACK/AFRICAN AMERICAN
31502,19998497,WHITE


In [ ]:
merged_data = pd.merge(
    merged_data,           # Use the DataFrame that contains medication and creatinine change data
    race_query,      # New demographics data
    on='subject_id',        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188476 entries, 0 to 7188475
Data columns (total 15 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   subject_id                 Int64         
 1   hadm_id                    Int64         
 2   drug                       object        
 3   ndc                        object        
 4   starttime                  datetime64[us]
 5   stoptime                   datetime64[us]
 6   creatinine_level           float64       
 7   previous_creatinine_level  float64       
 8   creatinine_change          float64       
 9   charttime                  datetime64[us]
 10  condition_satisfied        object        
 11  days_prescribed_ahead      float64       
 12  gender                     object        
 13  anchor_age                 Int64         
 14  race                       object        
dtypes: Int64(3), datetime64[us](3), float64(4), object(5)
memory usage: 843.2+ MB


In [ ]:
bmi_query = run_query(f"""
SELECT
    subject_id,
    AVG(CAST(result_value AS FLOAT64)) AS avg_bmi
FROM
    `physionet-data.mimiciv_hosp.omr`
WHERE
    LOWER(result_name) LIKE '%bmi%' AND
    subject_id IN {subject_ids}
GROUP BY
    subject_id
ORDER BY
    subject_id
""")
bmi_query

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,avg_bmi
0,10000032,18.537500
1,10000980,30.928767
2,10001338,25.200000
3,10002013,36.241237
4,10002430,25.900000
...,...,...
20004,19997538,35.633333
20005,19997752,27.600000
20006,19998330,28.185714
20007,19998497,19.400000


In [ ]:
merged_data = pd.merge(
    merged_data,           # Use the DataFrame that contains medication and creatinine change data
    bmi_query,
    on='subject_id',        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188476 entries, 0 to 7188475
Data columns (total 16 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   subject_id                 Int64         
 1   hadm_id                    Int64         
 2   drug                       object        
 3   ndc                        object        
 4   starttime                  datetime64[us]
 5   stoptime                   datetime64[us]
 6   creatinine_level           float64       
 7   previous_creatinine_level  float64       
 8   creatinine_change          float64       
 9   charttime                  datetime64[us]
 10  condition_satisfied        object        
 11  days_prescribed_ahead      float64       
 12  gender                     object        
 13  anchor_age                 Int64         
 14  race                       object        
 15  avg_bmi                    float64       
dtypes: Int64(3), datetime64[us](3), floa

In [ ]:
diagnosis_query = run_query(f"""
SELECT
    subject_id,
    hadm_id,
    MAX(CASE WHEN icd_code = 'N170' THEN 1 ELSE 0 END) AS N170,
    MAX(CASE WHEN icd_code = 'N171' THEN 1 ELSE 0 END) AS N171,
    MAX(CASE WHEN icd_code = 'N172' THEN 1 ELSE 0 END) AS N172,
    MAX(CASE WHEN icd_code = 'N178' THEN 1 ELSE 0 END) AS N178,
    MAX(CASE WHEN icd_code = 'N179' THEN 1 ELSE 0 END) AS N179,
FROM
    `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE
    icd_version = 10 AND subject_id IN {subject_ids}
GROUP BY
    subject_id, hadm_id
ORDER BY
    subject_id, hadm_id;
""")
diagnosis_query

<ipython-input-5-3b906ba92388>:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,N170,N171,N172,N178,N179
0,10000980,20897796,0,0,0,0,0
1,10000980,25911675,0,0,0,0,0
2,10000980,29659838,0,0,0,0,0
3,10002013,21763296,0,0,0,0,0
4,10002013,25442395,0,0,0,0,1
...,...,...,...,...,...,...,...
49938,19997538,22701415,0,0,0,0,0
49939,19997538,26704044,0,0,0,0,0
49940,19997752,29452285,0,0,0,0,1
49941,19999828,25744818,0,0,0,0,0


In [ ]:
merged_data = pd.merge(
    merged_data,           # Use the DataFrame that contains medication and creatinine change data
    diagnosis_query,
    on=['subject_id','hadm_id'],        # Join on subject_id
    how='left'              # Use left join to keep all entries from filtered_data
)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7188476 entries, 0 to 7188475
Data columns (total 21 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   subject_id                 Int64         
 1   hadm_id                    Int64         
 2   drug                       object        
 3   ndc                        object        
 4   starttime                  datetime64[us]
 5   stoptime                   datetime64[us]
 6   creatinine_level           float64       
 7   previous_creatinine_level  float64       
 8   creatinine_change          float64       
 9   charttime                  datetime64[us]
 10  condition_satisfied        object        
 11  days_prescribed_ahead      float64       
 12  gender                     object        
 13  anchor_age                 Int64         
 14  race                       object        
 15  avg_bmi                    float64       
 16  N170                       Int64    

In [ ]:
# File path to save the CSV file
csv_file_path = 'final_patient_data.csv'

# Save the final_result DataFrame to a CSV file
merged_data.to_csv(csv_file_path, index=False)

# Optionally, you can print a confirmation message
print(f"Final result saved to {csv_file_path}")


Final result saved to final_patient_data.csv


add diabetes info, histrory of aki, change medicaiton time to duration of medication
bmi of patient 7 days within medicaiton start time
